 # BAYESIAN OPTIMIZATION CV - XGBOOST CLASSIFIER

In [ ]:
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn import datasets
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from bayes_opt import BayesianOptimization
import pandas as pd
from datetime import datetime
from time import time

### GLOBAL VARIABLES

In [ ]:
DATAPATH = '../../../data/train_test/'
MODELPATH = '../../../models/xgboost/'
SEED = 47
NITER = 20
CV = 3
SCORE = 'roc_auc'
handlingnull = False
NJOBS = -1
NTHREADS= 4

### FUNCTIONS

In [ ]:
# Define Objective Function
def hyp_xgb(max_depth, subsample, colsample_bytree,min_child_weight, gamma):
    params = {
    'booster': booster,
    'n_estimators': n_estimators,
    'eta': eta,
    'objective': objective,
    'eval_metric':eval_metric, # Optional --> Use eval_metric if you want to stop evaluation based on eval_metric 
    'silent': 1
     }
    
    
    params['max_depth'] = int(round(max_depth))
    params['subsample'] = max(min(subsample, 1), 0)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['min_child_weight'] = int(min_child_weight)
    params['gamma'] = max(gamma, 0)

    
    
    cv_result = xgb.cv(params, xgtrain, num_boost_round=1000,verbose_eval=False, early_stopping_rounds=10, metrics = eval_metric, maximize=True, nfold=CV)
    return  max(cv_result['test-auc-mean'])

### LOAD DATASET

In [ ]:
train_features = pd.read_pickle(DATAPATH+'X_train.pkl')

In [ ]:
train_labels = pd.read_pickle(DATAPATH+'y_train.pkl')['target']

In [ ]:
train_features.shape

In [ ]:
train_labels.shape

In [ ]:
### create a DMatrix and handling Null values
if handlingnull:
    xgtrain = xgb.DMatrix(train_features, train_labels, missing=-9999)
else:
    xgtrain = xgb.DMatrix(train_features.values, train_labels.values)

### SET UP HYPERPARAMETERS

[xgboost params](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

In [ ]:
# ======== General Parameters ======= #

# Select the type of model to run at each iteration. gbtree or gblinear.
booster = 'gbtree'


# ======== Booster Parameters ======== # 

# Analogous to learning rate in GBM. 
# Typical final values to be used: 0.01-0.2
eta = 0.01


# Control the balance of positive and negative weights, useful for unbalanced classes. 
# A typical value to consider: sum(negative instances) / sum(positive instances)scale_pos_weight = 1
scale_pos_weight = int((len(train_labels) - np.sum(train_labels.values))/np.sum(train_labels.values))


# Learning Task Parameters

# This defines the loss function to be minimized. 
# - binary:logistic –logistic regression for binary classification, returns predicted probability (not class)
# - multi:softmax –multiclass classification using the softmax objective, returns predicted class (not probabilities)
#   you also need to set an additional num_class (number of classes) parameter defining the number of unique classes
# - multi:softprob –same as softmax, but returns predicted probability of each data point belonging to each class.
objective  = 'binary:logistic'


# The metric to be used for validation data.
# - rmse – root mean square error
# - mae – mean absolute error
# - logloss – negative log-likelihood
# - error – Binary classification error rate (0.5 threshold)
# - merror – Multiclass classification error rate
# - mlogloss – Multiclass logloss
# - auc: Area under the curve
eval_metric = 'auc'

### FIND NUM BOOST

In [ ]:
early_stopping_rounds = 10

In [ ]:
model=XGBClassifier(seed=SEED, booster=booster, objective=objective,  scale_pos_weight = scale_pos_weight, nthread=NJOBS)
xgb_param = model.get_xgb_params()
xgb_param['objective'] = objective

if USEGPU:
    xgb_param['tree_method'] = 'gpu_hist'
    xgb_param['gpu_id'] = 0


In [ ]:
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round = 1000, nfold = CV, metrics = eval_metric, early_stopping_rounds = early_stopping_rounds, seed = SEED)

In [ ]:
n_estimators = cvresult.shape[0]

In [ ]:
print("Best number of boosters: ", n_estimators)

### SET UP GRID SEARCH

In [ ]:
# Define search space
pds ={
    # Defines the minimum sum of weights of all observations required in a child.
    'min_child_weight':(14, 20),
    
    # A node is split only when the resulting split gives a positive reduction in the loss function. 
    # Gamma specifies the minimum loss reduction required to make a split.
    'gamma':(0, 5),
    
    # Denotes the fraction of observations to be randomly samples for each tree.
    'subsample':(0.5, 1),
    
    # Denotes the fraction of columns to be randomly samples for each tree.
    'colsample_bytree':(0.1, 1),
    
    # The maximum depth of a tree
    'max_depth': (5, 10)
}

### SEARCH BEST HYPERPARAMETERS

In [ ]:
optimizer = BayesianOptimization(hyp_xgb, pds, random_state=7)

In [ ]:
# Optimize
optimizer.maximize(init_points=5, n_iter=NITER)

#### Best parameter

In [ ]:
optimizer.max['params']

#### Best Score

In [ ]:
best_params = optimizer.max['params']

In [ ]:
best_params['n_estimators'] = n_estimators

### SAVE HYPERPARAMETERS AND RESULTS

In [ ]:
np.save(MODELPATH + 'hyperparameter/bayesianoptcv_xgboost_classifier_bestparams_d' + str(datetime.now().date()) + '.npy', best_params)